In [1]:
import tensorflow as tf

2024-06-09 19:33:24.809601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_generator = ImageDataGenerator()

In [21]:
directory_generator = img_generator.flow_from_directory(
    directory = "/tf/notebooks/data/train", # the folder where the class subfolders can be found
    target_size = (512,512), # The (height,width) of the produced images
    class_mode = "sparse", # Wether the target should be represented by an index or a dummy vector
    batch_size=16, # The batch size of the produced batches
    shuffle = True #Whether to shuffle after all files have been selected once
    #subset = "training"
)

directory_generator_val = img_generator.flow_from_directory(
    directory = "/tf/notebooks/data/test", # the folder where the class subfolders can be found
    target_size = (512,512), # The (height,width) of the produced images
    class_mode = "sparse", # Wether the target should be represented by an index or a dummy vector
    batch_size=16, # The batch size of the produced batches
    shuffle = True #Whether to shuffle after all files have been selected once
    #subset = "validation"
)

Found 15445 images belonging to 24 classes.
Found 4008 images belonging to 24 classes.


In [23]:
imgs, targets = next(iter(directory_generator))

In [24]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

modelconv = tf.keras.Sequential(
    [
     Conv2D(
         filters = 32,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu",
         input_shape = (imgs.shape[1], imgs.shape[2], imgs.shape[3]) # the input shape (height, width, channels)
     ),
     Conv2D(
         filters = 64,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     Conv2D(
         filters = 64,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     Conv2D(
         filters = 128,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     tf.keras.layers.Flatten(), # this layer turns multi-dimensional images into flat objects
     tf.keras.layers.Dense(24, activation="softmax")
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-09 19:53:37.705968: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: UNKNOWN ERROR (34)


In [25]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

modelconv.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

modelconv.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │     3,145,752 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,275,928 (12.50 MB)

 Trainable params: 3,275,928 (12.50 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
modelconv.fit(directory_generator, epochs = 10, validation_data=directory_generator_val)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15445/15445 ━━━━━━━━━━━━━━━━━━━━ 943s 61ms/step - loss: 3.8401 - sparse_categorical_accuracy: 0.1103 - val_loss: 2.9774 - val_sparse_categorical_accuracy: 0.1035
Epoch 2/10
15445/15445 ━━━━━━━━━━━━━━━━━━━━ 904s 59ms/step - loss: 2.9872 - sparse_categorical_accuracy: 0.1108 - val_loss: 3.0186 - val_sparse_categorical_accuracy: 0.1075
Epoch 3/10
15445/15445 ━━━━━━━━━━━━━━━━━━━━ 923s 60ms/step - loss: 2.9603 - sparse_categorical_accuracy: 0.1110 - val_loss: 2.9725 - val_sparse_categorical_accuracy: 0.0988
Epoch 4/10
15445/15445 ━━━━━━━━━━━━━━━━━━━━ 916s 59ms/step - loss: 2.9393 - sparse_categorical_accuracy: 0.1141 - val_loss: 3.0045 - val_sparse_categorical_accuracy: 0.1073
Epoch 5/10
15445/15445 ━━━━━━━━━━━━━━━━━━━━ 892s 58ms/step - loss: 2.9033 - sparse_categorical_accuracy: 0.1302 - val_loss: 3.0089 - val_sparse_categorical_accuracy: 0.1088
Epoch 6/10
15445/15445 ━━━━━━━━━━━━━━━━━━━━ 898s 58ms/step - loss: 2.8691 - sparse_categorical_accuracy: 0.1350 - val_loss: 3.1385 - val_sparse_ca